# Soccer League Season

## Runs an entire season of games for a professional soccer league

Every new programming language I have ever learned I would write the same program, which was the very first program of my own creation I ever coded. It reproduced the play of a dice baseball game, where each roll of 2 6-sided dice gave the result of one at-bat.

By the time I came to learn Python, I wanted to do something different. So I developed an algorithm to reproduce the results of Premiere League matches fairly realistically. That was written as a Python script. This is an adaptation of that program as a Jupyter notebook. 

This notebook is fairly well documented, and the mark-up cells point out various Python features that may help you learn the basics, just as writing this program helped me learn them. Often the best way to learn a programming language is to explore other people's code. The goal here is not to write the most efficient or _pythonic_ program, but to incorporate many different elements of the language.

#### Game play algorithm:
* Each match is divided into 30 3-minute intervals (15 per half)
* Each interval has a shot probability for each team
    * Home team gets first chance to have a shot in an interval (only one shot allowed per interval) - this conveys home pitch advantage because if they take a shot, the opposition does not get a chance in that interval
    * Each team's odds of taking a shot are determined by a combination of their offensive and opponent's defensive ratings
* If there is a shot, it has an on-target probability determined only by the shooting team's offensive rating
* If the shot is on goal, it has a probability to get past the keeper based on the keeper's rating
* Best of all, **no VAR**!


In [ ]:
# Import libraries

import sys                           # Only used to debug
import numpy as np
import pandas as pd
import random                        # Used to randomize game play
import matplotlib.pyplot as plt      # Used to plot a seasonal graph of the evolving table in last notebook cell
# random.seed(a=3) # For testing reproducability and debugging
from operator import itemgetter      # Used to aid sorting
from scipy.optimize import curve_fit # Used to calculate probabilities used in the game

## Functions defined

Functions used by this notebook:

* `func_soccer` = the function that defines how probabilities for shots taken, shots on goal and goals scored vary with team ratings
* `play_match`  = executes the match
* `print_match_line` = prints out the scoreline for a match
* `update_table` = updates the league table after each week

#### Python elements to notice:

`func_soccer`:
* There are no lines in the function - the equation is actually stated as the argument to the `return` command

`play_match`:
* In each 3-minute interval, nested `if` statements to step through:
    * Is there a shot taken?
    * Is it on goal?
    * Does it get past the keeper?
* `random.random()` generates a random number $n: 0≤n<1$ chosen from a uniform (flat) distribution
* Function returns a pair of lists - stats for each team

`print_match_line`:
* Function makes sure the grammar is correct in the string printed.

`update_table`
* Makes heavy use of the increment operator `+=`.
* The very particular "last 5 match results" string is built here

In [ ]:
### Function to fit thru probabilities y = a - b*(1-exp(-k*x))
def func_soccer(x, a, b, k):
    """
    Required inputs:
    x = probability index for scoring
    a,b,k = curve fit parameters determined in the block of code below this function
    
    Output:
    a - b/k * (1 - exp(-kx))
    """
    return a - b/k * (1 - np.exp(-k * x))

### Function that plays a match
def play_match(home_team,away_team,show_scores=False):
    """
    Required inputs:
    home_team = [int] the index number for the home team
    away_team = [int] the index number for the away team

    Optional inputs:
    show_scores = [bool] should individual scores be printed? [default = False]

    Outputs:
    home_stats = [list]: home team score [int], shots on target [int], total shots [int], and score times [str]
    away_stats = [list]: away team score [int], shots on target [int], total shots [int], and score times [str]
    """
    
    # Find the perturbed ratings for the two teams
    home_shotp = func_soccer(dcafe[away_team]-ocafe[home_team]+2, *co_shot)
    home_targp = func_soccer(3-ocafe[home_team], *co_targ)
    home_keepp = func_soccer(kcafe[away_team]-1, *co_keep)
    
    away_shotp = func_soccer(dcafe[home_team]-ocafe[away_team]+2, *co_shot)
    away_targp = func_soccer(3-ocafe[away_team], *co_targ)
    away_keepp = func_soccer(kcafe[home_team]-1, *co_keep)
    
    # Set up the "scoreboard" for the start of the match
    home_goals_string = team_data[home_team]['abbr'] + ":"
    away_goals_string = team_data[away_team]['abbr'] + ":"
    home_shotc, home_targc, home_score = 0, 0, 0
    away_shotc, away_targc, away_score = 0, 0, 0

    # Loop through the 3-minute time intervals        
    for interval in range(0, 30):
        cheer = False    # This is an indicator of whether the home team scores 

        # Does home team score? (Having first chance is home field advantage)          
        if (random.random() < home_shotp):            # Is probability of a shot realized?
            home_shotc += 1 
            if (random.random() < home_targp):        # Is probability of a shot on target realized?
                home_targc += 1
                if (random.random() < home_keepp):    # Is probability of a shot in the net realized?
                    home_score += 1
                    cheer = True
                    goal_time = int(random.uniform(0,3))+interval*3+1
                    home_goals_string += " " + str(goal_time) + "'" 
                    if show_scores:
                        print(f"{teams[home_team]} @ {goal_time}'")

        # If not, does away team score?     
        if (random.random() < away_shotp) and not cheer:  # Is probability of a shot realized?
            away_shotc += 1 
            if (random.random() < away_targp):            # Is probability of a shot on target realized?
                away_targc += 1
                if (random.random() < away_keepp):        # Is probability of a shot in the net realized?
                    away_score += 1
                    goal_time = int(random.uniform(0,3))+interval*3+1
                    away_goals_string += " " + str(goal_time) + "'"
                    if show_scores:
                        print(f"{teams[away_team]} @ {goal_time}'")

    # End of match - gather up the stats
    home_stats = [home_score,home_targc,home_shotc,home_goals_string]
    away_stats = [away_score,away_targc,away_shotc,away_goals_string]
    
    return home_stats,away_stats

### Function that prints out the match scoreline
def print_match_line(home_stats,away_stats):
    print("FT:",team_data[home_team]['name'],home_stats[0],"-",away_stats[0],team_data[away_team]['name'])
    
    # This one needs some grammar adjustments for singular numbers
    if home_stats[2] == 1:
        home_shots = str(home_stats[2])+" shot, "
    else:
        home_shots = str(home_stats[2])+" shots, "
    if home_stats[1] == 1:
        home_sog = str(home_stats[1])+" shot on goal; "
    else:
        home_sog = str(home_stats[1])+" shots on goal; "
    if away_stats[2] == 1:
        away_shots = str(away_stats[2])+" shot, "
    else:
        away_shots = str(away_stats[2])+" shots, "
    if away_stats[1] == 1:
        away_sog = str(away_stats[1])+" shot on goal."
    else:
        away_sog = str(away_stats[1])+" shots on goal."

    # Handle if there are no scores by a team
    if home_stats[0] == 0:
        home_scores = home_stats[3]+" "
    else:
        home_scores = home_stats[3]+", "
    if away_stats[0]== 0:
        away_scores = away_stats[3]+" "
    else:
        away_scores = away_stats[3]+", "
        
    print("> "+home_scores+home_shots+home_sog+away_scores+away_shots+away_sog)

    return

### Function to update table
def update_table(home_team,away_team,home_stats,away_stats):
    """
    Required inputs:
    home_team = [int] the index number for the home team
    away_team = [int] the index number for the away team
    home_stats = [list]: home team score [int], shots on target [int], total shots [int], and score times [str]
    away_stats = [list]: away team score [int], shots on target [int], total shots [int], and score times [str]
    
    Outputs:
    None
    
    But global table_* variables are updated directly
    
    """
    
    if (home_stats[0] > away_stats[0]):
        table_w[home_team] += 1
        last_five[home_team] = "W" + last_five[home_team] 
        if (len(last_five[home_team]) > 5):
            last_five[home_team] = last_five[home_team][:5]
        table_l[away_team] += 1
        last_five[away_team] = "L" + last_five[away_team]
        if (len(last_five[away_team]) > 5):
            last_five[away_team] = last_five[away_team][:5]
    elif (home_stats[0] < away_stats[0]):
        table_w[away_team] += 1
        last_five[away_team] = "W" + last_five[away_team]
        if (len(last_five[away_team]) > 5):
            last_five[away_team] = last_five[away_team][:5]
        table_l[home_team] += 1
        last_five[home_team] = "L" + last_five[home_team]
        if (len(last_five[home_team]) > 5):
            last_five[home_team] = last_five[home_team][:5]        
    else:
        table_d[home_team] += 1
        last_five[home_team] = "D" + last_five[home_team]
        if (len(last_five[home_team]) > 5):
            last_five[home_team] = last_five[home_team][:5]        
        table_d[away_team] += 1
        last_five[away_team] = "D" + last_five[away_team]
        if (len(last_five[away_team]) > 5):
            last_five[away_team] = last_five[away_team][:5]

    table_gf[home_team] += home_stats[0]
    table_ga[away_team] += home_stats[0]
    table_gf[away_team] += away_stats[0]
    table_ga[home_team] += away_stats[0]
    table_gd[home_team] = table_gf[home_team] - table_ga[home_team]
    table_gd[away_team] = table_gf[away_team] - table_ga[away_team]
    table_pts[home_team] = 3 * table_w[home_team] + table_d[home_team]
    table_pts[away_team] = 3 * table_w[away_team] + table_d[away_team]
    
    return


## Game parameter settings

* The game is built around three ratings for each team, and three probabilities based on those ratings
* The boolean flags at the start are to control what gets printed out, and whether the program pauses after each game or each week.
* `pertmag` allows the team rankings to meander throughout the season, making results less predictable

#### Python elements to notice:
* Tuples and lists are initialized here, including lists full of zeroes
* The `scipy.optimize` function `curve_fit` returns two objects - a 3-element vector and a 3x3 array (the latter is not used here)

In [ ]:
#######################################################################
#######################################################################
match_wait = False   # Requires keyboard input to move to next match
week_wait = False    # Requires keyboard input to play the next week
week_scores = False   # To print the scores from each game each week
week_table = False    # To print the updated table at the end of each week
show_scores = False  # To print each goal as it happens

#######################################################################
# Probabilities for scoring associated with integer rating values
#>>> Example of "tuples" in Python ()
shotp = (0.31, 0.35, 0.41, 0.49, 0.59) # index is (drate - orate + 2)
targp = (0.26, 0.32, 0.38)             # index is (3 - orate)
keepp = (0.24, 0.31, 0.36)             # index is (krate - 1)
pertmag = 0.06                         # Allows for team fortunes to drift during season - set to 0.0 for no random changes in ratings over time

#######################################################################
# Calculate curve fits that allow for non-integer values of the three probability indices
# This allows for a Brownian random creep to be applied to the ratings for each team as the season evolves
xdata = (0.0, 1.0, 2.0, 3.0, 4.0) # The indices for shotp
popt, pcov = curve_fit(func_soccer, xdata, shotp)
co_shot = popt                    # a, b and k values to best match shotp
xdata = (0.0, 1.0, 2.0)           # The indices for targp
popt, pcov = curve_fit(func_soccer, xdata, targp)
co_targ = popt                    # a, b and k values to best match targp    
xdata = (0.0, 1.0, 2.0)           # The indices for keepp
popt, pcov = curve_fit(func_soccer, xdata, keepp)
co_keep = popt                    # a, b and k values to best match keepp 

# Set up lists of perturbations to team ratings that can evolve
# in a random Brownian manner thru the season
#>>> Empty lists in Python []
opert = [0] * 20
dpert = [0] * 20
kpert = [0] * 20 


## Team parameter settings

Here the teams are defined.
Team data are stored as a list [] of dictionaries {} - one dictionary per team.

_All teams in this imaginary league are located in Ohio and Indiana (because, why not? They have funny place names.)_

Feel free to substitute your own teams and ratings - for scheduling to work, need an even number of teams.

#### Python elements to notice:
* Each team has a dictionary with the same _keys_ but different values assigned to each key
* `nteams` counts the number of teams - this is used to set up the schedule
* There is a check that there is an even number of teams - the cell is exited otherwise

In [ ]:
#######################################################################
# name = Full team name (a stinging mockery of MLS's apeing of names 
#                  from traditional European and South American clubs)
# abbr = 2-3 letter abbreviation of team name
# col1 = primary team color (hex RGB code)
# col2 = secondar team color (hex RGB code)
# orate = offensive rating (1 high, 3 low)
# drate = defensive rating (1 high, 3 low)
# krate = goalkeeper rating (1 high, 3 low)
team_data = [
    {'name':"Deportes Akron",'abbr':"AKR",'col1':"#C81B17",'col2':"#EFDBB2",'orate':1,'drate':1,'krate':3},
    {'name':"Atlético Bloomington",'abbr':"BLM",'col1':"#EF0107",'col2':"#DB0007",'orate':2,'drate':3,'krate':2},
    {'name':"Canton(ham) Hot Pockets",'abbr':"CAN",'col1':"#0057B8",'col2':"#EEEEEE",'orate':2,'drate':3,'krate':2},
    {'name':"Daytonoord",'abbr':"DAY",'col1':"#6C1D45",'col2':"#99D6EA",'orate':3,'drate':3,'krate':1},
    {'name':"Evans Villa",'abbr':"EVN",'col1':"#0070B5",'col2':"#D11524",'orate':3,'drate':3,'krate':3},
    {'name':"Eintracht Ft Wayne",'abbr':"FTW",'col1':"#039446",'col2':"#D1D3D4",'orate':0.6,'drate':1,'krate':2},
    {'name':"Gary Saint-Germain",'abbr':"GAR",'col1':"#1B458F",'col2':"#C4122E",'orate':1.5,'drate':2,'krate':1},
    {'name':"Indianapiakos",'abbr':"IND",'col1':"#003399",'col2':"#003399",'orate':3,'drate':1,'krate':2},
    {'name':"Lafayette Albion",'abbr':"LAF",'col1':"#CC0000",'col2':"#000000",'orate':3,'drate':2,'krate':2},
    {'name':"Celta Lima",'abbr':"LIM",'col1':"#0E63AD",'col2':"#FFFFFF",'orate':3,'drate':3,'krate':3},
    {'name':"Marionense",'abbr':"MAR",'col1':"#003090",'col2':"#FDBE11",'orate':2,'drate':2,'krate':0.6},
    {'name':"Lokomotív Muncie",'abbr':"MUN",'col1':"#C8102E",'col2':"#FFFFFF",'orate':1,'drate':1,'krate':2},
    {'name':"Sparta Sandusky",'abbr':"SAN",'col1':"#6CABDD",'col2':"#1C2C5B",'orate':1,'drate':0.6,'krate':2},
    {'name':"Borussia South Bend",'abbr':"SBN",'col1':"#DA291C",'col2':"#FBE122",'orate':2.5,'drate':1.5,'krate':1},
    {'name':"Springfield Wednesday",'abbr':"SPR",'col1':"#241F20",'col2':"#41B6E6",'orate':2,'drate':2,'krate':2},
    {'name':"Maccabi Terra Haute",'abbr':"THT",'col1':"#130C0E",'col2':"#D71920",'orate':2,'drate':3,'krate':2},
    {'name':"Dinamo Toledo",'abbr':"TOL",'col1':"#132257",'col2':"#FFFFFF",'orate':1,'drate':1,'krate':2},
    {'name':"River Wabash",'abbr':"WAB",'col1':"#FBEE23",'col2':"#ED2127",'orate':2,'drate':3,'krate':3},
    {'name':"Youngstown Boys",'abbr':"YNG",'col1':"#7A263A",'col2':"#1BB1E7",'orate':3,'drate':2,'krate':2},
    {'name':"Zenit Zainsville",'abbr':"ZNV",'col1':"#FDB913",'col2':"#231F20",'orate':2,'drate':3,'krate':2}]

#######################################################################
nteams = len(team_data)  # By default, a season consists of 2*(nteams-1) matches; each team plays every other team twice (home and away)

if nteams % 2:
    print("Must have an even number of teams in the league")
    sys.exit(1)
else:
    tcount = list(range(nteams))


## Set up season schedule

This cell sets up the season schedule.
Since every team plays every other team twice, once at home and once away, there is a lot of symmetry we can take advantage of.
The scheduling algorithm here is a pair of "round robin" schedules, each built using a "circle" rotation around the list of teams.
The last step swaps the home and away teams every second week. 
This ensures most teams alternate most weeks between home and away games, and never have more than two home or away games in a row.

#### Python elements to notice:
* Lines 7-8: Use of simple _list comprehensions_ to initialize the arrays `team_home` and `team_away`
* Lines 11-12: Define first inner list by converting a range into a list with the `list()` function
* Line 16: Create null lists `new_home` and `new_away` that are then grown into complete lists using the `append()` method
* Line 37-38: Looping by twos to swap home and away lists - note how swapping can be done in one line with the multiple assign feature

In [ ]:
# Create empty arrays to house schedule    
weeks = 2 * (nteams-1)       # Number of weeks in season
games = int(nteams/2)        # Number of matches per week
#week_home = [[0] * games for i in range(weeks)]
#week_away = [[0] * games for i in range(weeks)]
# These arrays just hold the indices 0-19 pointing to the list elements in team_data
team_home = [[0] * games for i in range(weeks)]  # Array (list of lists) of the home teams for each match of each week
team_away = [[0] * games for i in range(weeks)]  # Array (list of lists) of the away teams for each match of each week
    
# Set up first week of pattern - teams 0 thru 19 are in the same order as in team_data
team_home[0] = list(range(0, games))
team_away[0] = list(range(nteams-1, games-1, -1))

# Loop thru weeks, keep team 0 in same slot, rotate others thru
for week in range(0, weeks-1):
    new_home, new_away = [], [] # Clear out the week's lists of teams

    # Set up the teams in each game by sliding the slots from the previous week
    # Game #1
    new_home.append(team_home[week][0])
    new_away.append(team_home[week][1])

    # Games #2-#9
    for game in range(1, games-1):
        new_home.append(team_home[week][game+1])
        new_away.append(team_away[week][game-1])

    # Game #10
    new_home.append(team_away[week][9])
    new_away.append(team_away[week][8])

    # Place new week's pairings into the schedule matrix        
    team_home[week+1] = new_home
    team_away[week+1] = new_away

# Invert home and away slots every other week     
for week in range(0, weeks, 2):
    team_home[week], team_away[week] = team_away[week], team_home[week]


## Season stats and standings

We want to be able to print standings tables, and at the end of the season plot the evolution of teams' standings.
These lists will hold that information.

#### Python elements to notice:
* A separate list is created for each column of the standings table - this makes printing easier later
* Rank and points at end of each week are created as empty lists of lists with _list comprehensions_

In [ ]:
# Set up table statistics
table_w = [0] * nteams
table_d = [0] * nteams
table_l = [0] * nteams
table_gf = [0] * nteams
table_ga = [0] * nteams
table_gd = [0] * nteams
table_pts = [0] * nteams
table_rank = [0] * nteams

# Keep track of evolution of stats through season for each team
weekly_rank = [[0] * nteams for i in range(weeks)]
weekly_pts = [[0] * nteams for i in range(weeks)]
last_five = [""] * nteams


## Playing the season

This cell loops through weeks of the season and games of each week.

#### Python elements to notice:
* A strength of Jupyter notebooks is that a program like this can be cut into cells that can be executed separately. However, loops cannot span multiple cells - this is something of a weakness
* Lines 6-8: Uses the iterator function `zip` to add together lists element-wise. We also create lists from all the dictionary values with the same key
* Boolean variables are queried with `if` statements that act like switches, turining on and off game reporting features.
* Most of the game play occurs within defined fuctions from cells above
* Line 38: `zip` is used again to build a big tuple of the league table data. This allows the sort to rearrange entire rows of the table, much as sorts do by defalt in Excel spreadsheets
* Line 41: A double sort is performed so that the teams can be listed in order of their standings
    * The inner sort, performed first, sorts by goal differential, which is actually the first tiebreaker
    * The outer sort, performed last, is on points, and is the main sort since it is done last
* Line 42: The list of comma-separated variables on the LHS are lists extracted from the `zip` iterable, corresponding to columns of sorted data in the league standings table.
* Lines 52-53: The formatted print statement defines fixed widths for each column, ensuring the table looks good.
* Lines 63-65: Perturbed ratings are updated randomly using `random.uniform` spanning ±`pertmag`
* Line 68: The `input` function waits for user input before continuning to run.

#### To do:
* Add in a way to have "extra time", and shots in extra time.


In [ ]:
# Loop through weeks ####################################################
for week in range(weeks):
    print("Week: ",int(week+1))

    # Update perturbed ratings (does nothing for first week)   
    ocafe = [sum(i) for i in zip([team_data[i]['orate'] for i in range(nteams)],opert)]
    dcafe = [sum(i) for i in zip([team_data[i]['drate'] for i in range(nteams)],dpert)]
    kcafe = [sum(i) for i in zip([team_data[i]['krate'] for i in range(nteams)],kpert)]

    # For fun, shuffle sequence of matches for the week (simulates TV scheduling)
    gseq = [i for i in range(games)]
    random.shuffle(gseq)
    # Loop through games ################################################
    for game in range(0, games):

        # Set up bookkeeping for game
        home_team = team_home[week][gseq[game]]
        away_team = team_away[week][gseq[game]]
        
        print("  Game",int(game+1),"-",team_data[home_team]['name'],"vs",team_data[away_team]['name'])

        # Play the match! <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        # returned lists have 3 elements: score, shots on target, total shots, and score times
        home_stats,away_stats = play_match(home_team,away_team,show_scores=show_scores)
        
        # Print the scoreline from the match
        if week_scores:
            print_match_line(home_stats,away_stats)
            print(" ")    

        ##### Match over - update table stats 
        update_table(home_team,away_team,home_stats,away_stats)

        if match_wait:
            input("Hit [enter] to continue to next match.")
            
    ##### The week is done - updated the table
    table_tup = zip(team_data,table_w,table_d,table_l,table_pts,table_gf,table_ga,table_gd,last_five,tcount)

    # Do a complex sort, primarily on points, secondarily on goal difference 
    table_sort = sorted(sorted(table_tup, key=itemgetter(5), reverse=True), key=itemgetter(4), reverse=True)
    t_team,t_win,t_draw,t_loss,t_pts,t_gf,t_ga,t_gd,t_l5,t_tc = zip(*table_sort)


    # Print table
    if week_table:
        print(" ")
        print("Week: ",int(week+1))
        print("   Team                       W   D   L  Pts  GF  GA  Dif")
    for x in range(0, nteams):
        if week_table:
            print("{0:2} {1:25} {2:2d}  {3:2d}  {4:2d}  {5:3d} {6:3d} {7:3d}  {8:+3d} {9}".format(x+1,
                t_team[x]['name'],t_win[x],t_draw[x],t_loss[x],t_pts[x],t_gf[x],t_ga[x],t_gd[x],t_l5[x]))
        weekly_rank[week][t_tc[x]] = x+1
        weekly_pts[week][t_tc[x]] = t_pts[x]

    print(" ")    
    print("-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
    print(" ")
    
    # Update perturbations to team ratings
    for x in range(0, nteams): 
        opert[x] += random.uniform(-pertmag,pertmag)
        dpert[x] += random.uniform(-pertmag,pertmag)
        kpert[x] += random.uniform(-pertmag,pertmag)
   
    if week_wait:
        input("Hit [enter] to continue to next week.")
#    sys.exit()
    
print(" ")


## Plot season history

Generate a plot showing how all the teams accumulated points, week by week. 
Shows how leads and fortunes evolve throughout the season.

#### Python elements to notice:
* This cell mostly showcases the `pyplot` functions from `matplotlib`

In [ ]:
# Plot a graph of season's week-by-week team performance
plt.figure(figsize=(11,8.5))

pmax = max(weekly_pts[-1])   # Max points used to scale the plot
vtint = 0.05 * (pmax - 25)

# Set plot domain 
plt.xlim((0, week+1.5))
plt.ylim((-2, pmax+4))

plt.tick_params(axis='y',left='on',right='on',labelleft='on',labelright='on')

# Set up the labels for the weeks on the x-axis
x_ticks = [1]
x_int = int(week/7)
[x_ticks.append(i) for i in range(x_int,week+1,x_int)]
if x_ticks[-1] != week+1:
    x_ticks.append(week+1)
elif x_ticks[-1] == week:
    x_ticks.pop()
    x_ticks.append(week+1)
plt.xticks(x_ticks)

plt.vlines((week+2)/2,-2,pmax+4,colors="#606060") # Midpoint of season marked

# Draw lines for each team's progression through season
for t in range(0, nteams):
    pseries = [x[t] for x in weekly_pts]
    df=pd.DataFrame({'x': range(1,39), 'y': pseries })  # Using Pandas to make a dataframe that will be straightforward to plot
    
    # Plot the line on the graph
    plt.plot('x','y','-',data=df,color=team_data[t]['col2'],linewidth=3)
    plt.plot('x','y','--',data=df,color=team_data[t]['col1'],linewidth=2)
    
    ### Build the legend manually
    # Plot the team's abbreviation in the team colors
    plt.text(3.26,pmax-0.04-vtint*(weekly_rank[-1][t]-1),team_data[t]['abbr'],va='center',ha='right',fontsize=11,color=team_data[t]['col2'],weight='normal')
    plt.text(3.3,pmax-vtint*(weekly_rank[-1][t]-1),team_data[t]['abbr'],va='center',ha='right',fontsize=11,color=team_data[t]['col1'],weight='normal')
    
    # Plot the team's final points in the team colors
    plt.text(6.96,pmax-0.04-vtint*(weekly_rank[-1][t]-1),table_pts[t],va='center',ha='left',fontsize=11,color=team_data[t]['col2'],weight='normal')
    plt.text(7.0,pmax-vtint*(weekly_rank[-1][t]-1),table_pts[t],va='center',ha='left',fontsize=11,color=team_data[t]['col1'],weight='normal')
    
    # Plot a line sample in the legend
    plt.plot([4.00,6.33],[pmax-vtint*(weekly_rank[-1][t]-1),pmax-vtint*(weekly_rank[-1][t]-1)],color=team_data[t]['col2'],linestyle='-',linewidth=3)
    plt.plot([4.00,6.33],[pmax-vtint*(weekly_rank[-1][t]-1),pmax-vtint*(weekly_rank[-1][t]-1)],color=team_data[t]['col1'],linestyle='--',linewidth=2)
    
plt.ylabel('Points',fontsize=12)
plt.xlabel('Week',fontsize=12)
plt.title('Week-by-Week Progression of Season',fontsize=16)

plt.show()

# Print table
print(" ")
print("Week: ",int(week+1))
print("   Team                       W   D   L  Pts  GF  GA  Dif")
for x in range(0, nteams):
    print("{0:2} {1:25} {2:2d}  {3:2d}  {4:2d}  {5:3d} {6:3d} {7:3d}  {8:+3d} {9}".format(x+1,
        t_team[x]['name'],t_win[x],t_draw[x],t_loss[x],t_pts[x],t_gf[x],t_ga[x],t_gd[x],t_l5[x]))
    weekly_rank[week][t_tc[x]] = x+1
    weekly_pts[week][t_tc[x]] = t_pts[x]
